In [1]:
import os
import sys
import glob
import traceback
from datetime import datetime
import pandas as pd
from plotly.graph_objs import Scatter,Layout
import plotly
import plotly.offline as py
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots
plotly.offline.init_notebook_mode(connected=True)
import plotly.express as px
import gezi
from gezi import tqdm 
import tensorflow as tf
flags = tf.app.flags
FLAGS = flags.FLAGS
flags.DEFINE_string('f', None, '')
FLAGS(sys.argv)
from projects.feed.rank.src.evaluate import evaluate_df as eval
import pymp
from multiprocessing import Pool, Manager, cpu_count 
import qgrid

In [2]:
mark = 'video'
root = f'/search/odin/publicData/CloudS/chenghuige/rank/data/{mark}_hour_sgsapp_v1/exps/'
root = f'/home/gezi/tmp/rank/exps/{mark}/monitor'
# root = f'/search/odin/mkyuwen/rank_0522/exps/video/libowei/' # mkyuwen
# root = f'/search/odin/mkyuwen/rank_rel_0709/exps/video/libowei' # mkyuwen
root = f'/search/odin/mkyuwen/rank_rel_0709/exps/video/libowei' # mkyuwen
# root = f'/search/odin/mkyuwen/new_pikachu_0812/rank_0812/exps/video/libowei' # mkyuwen
# root = f'/home/gezi/tmp/rank/data/{mark}_hour_sgsapp_v1/exps/v20'
# root = f'/home/gezi/tmp/rank/data/{mark}_hour_sgsapp_v1/exps/v21'
day_ = 20191224
days = 100
day = gezi.DateTime(str(day_), day=True)

In [3]:
models = [os.path.basename(x) for x in glob.glob(f'{root}/*')]
print(models)
# names = [
#   'fm',
#   'fm.deeponly',
# ]

# SHOW_MASK_EXP = 
# SHOW_
def mktest_filter(models):
    models_filter = []
    for model in models:
#         if "0525" not in model and "8" not in model: #or "Fix" in model: #or "Mask" in model:
        if False:
            continue
        else:
            models_filter.append(model)
    return models_filter
# # mkyuwen
# models_ = mktest_filter(models)
# models_

models


models_filter = mktest_filter(models)
models = models_filter

['model.123', 'baseline_baili_restart_0601', 'model.bk.0625', 'model.bk.0727', 'rel_Kw_betterMASK_saveMITATKWsnMITATsingle_mTo51_now2vUp_avgPooling_restart_0601.continue', 'rel_Kw_betterMASK_saveMITATKWsnMITATsingle_mTo51_now2vUp_avgPooling_restart_0601.v25.bk', 'rel_Kw_betterMASK_saveMITATKWsnMITATsingle_mTo51_now2vUp_avgPooling_restart_0601.v25.goon.bk', 'rel_Kw_betterMASK_saveMITATKWsnMITATsingle_mTo51_now2vUp_avgPooling_restart_0601', 'rel_Kw_betterMASK_mTo38_now2vUp_avgPooling_restart_0601', 'rel_Kw_betterMASK_mTo34_now2vUp_avgPooling_restart_0601', 'rel_Kw_betterMASK_mTo37_now2vUp_avgPooling_restart_0601', 'rel_Kw_betterMASK_mTo33_now2vUp_avgPooling_restart_0601', 'cp_model_bk.sh', 'model.rel_new_mask.bk.0815']


In [4]:
dfs = Manager().list()
files = []
for model in models:
#   files += glob.glob(f'{root}/{model}/infos_day/*/metrics_impression.csv')
  files += glob.glob(f'{root}/{model}/infos_day/*/metrics.csv')
ps = min(len(files), cpu_count())
with pymp.Parallel(ps) as p:
  for i in tqdm(p.range(len(files))):
#     print (files[i])
    dfs += [pd.read_csv(files[i])]
dfs = list(dfs)
dfs
print(len(dfs))









































343


In [5]:
df = pd.concat(dfs)

In [6]:
df[df.model=='base'].sort_values('date')[['date', 'group/ndcg_click']].tail()

,date,group/ndcg_click


In [7]:
# df = df[df.model.isin(['deeponly-udh-simple', 'deeponly-udh-simple-leaky'])]
# df = df[df.model.isin([x for x in models if 'torch' in x])]
df = df[~df.model.isin(['online'])]

In [8]:
df['sqrt/auc'] = (df['auc'] * df['click/time_auc']) ** 0.5
df['group/sqrt/auc'] =  (df['group/auc'] * df['group/click/time_auc']) ** 0.5
df['quality/sqrt/auc'] = (df['quality/auc'] * df['quality/click/time_auc']) ** 0.5
df['quality/group/sqrt/auc'] =  (df['quality/group/auc'] * df['quality/group/click/time_auc']) ** 0.5
df['rmse'] = 1. - df['mse'] ** 0.5
df['click/rmse'] = 1. - df['click/mse'] ** 0.5

In [9]:
cols = ['date', 'model','sqrt/auc', 'group/sqrt/auc', 'quality/sqrt/auc', 'quality/group/sqrt/auc', 'inv_rate']
if mark == 'video':
  cols += ['finish_auc', 'vtime_auc']
# qgrid.show_grid(df[cols].sort_values(['date', 'group/sqrt/auc'], ascending=[False, False]))
metrics = [ 'hours',
           'Click/group/auc',  'Click/group/auc2', 'Dur/group/click/time_auc',
           'group/auc', 'group/auc2', 'cold/group/auc',
            'group/sqrt/auc',
           'sqrt/auc', 
           
           'auc', 'clickmids/auc', 'cold/auc',
           'quality/group/auc',
           'group/click/time_auc', 'quality/group/click/time_auc', 'rmse', 'click/rmse', 
           'rcr_auc', 'vtime_auc', 'ptime_auc',   
           'group/ndcg3_click', 'group/ndcg_click', 
           'stats/click_ratio','stats/num_instances','stats/pos_ratio',
           'loss/click', 'loss/dur',
           
          ]
gezi.line(df, x='date', y=metrics, color='model')

In [10]:
%%html
<style>
.output_wrapper, .output {
    height:auto !important;
    max-height:500000px;  /* your desired max-height here */
}
.output_scroll {
    box-shadow:none !important;
    webkit-box-shadow:none !important;
}
</style>

In [11]:
# df[['date', 'model', 'auc', 'group/auc', 'cold/auc', 'clickmids/auc',
#     'click/time_auc','group/click/time_auc',
#    ]].sort_values(['date', 'group/auc'], ascending=[False, False]).head(10)

In [12]:
# df[[
#     'date', 'model','quality/auc', 'quality/group/auc', 
#     'quality/click/time_auc', 'quality/group/click/time_auc'
# ]].sort_values(['date', 'quality/click/time_auc'], ascending=[False, False]).head(10)

In [13]:
# df[[
#       'date', 'model', 'group/click/top1_dur_rate', 'group/click/top3_dur_rate', 'group/top1_click_rate', 'group/top3_click_rate'
# ]].sort_values(['date', 'group/top3_click_rate'], ascending=[False, False]).head(10)

### online 是线上tfrecord落盘数据直接计算 base是指定的模型比如16尾号的验证 有可能并不是当时线上全量作用于abtest 4,5,6的模型 
### 这里的验证都是针对abtest 4,5,6 小写click表示只针对正样本 Click/表示点击塔输出结果的指标 Dur/表示时长塔

In [14]:
df[['model', 'hours', 'date']].head()

,model,hours,date
0,rel_Kw_betterMASK_saveMITATKWsnMITATsingle_mTo...,22,20200601
0,rel_Kw_betterMASK_saveMITATKWsnMITATsingle_mTo...,22,20200619
0,rel_Kw_betterMASK_saveMITATKWsnMITATsingle_mTo...,23,20200628
0,rel_Kw_betterMASK_saveMITATKWsnMITATsingle_mTo...,24,20200610
0,rel_Kw_betterMASK_saveMITATKWsnMITATsingle_mTo...,24,20200716


In [15]:
# df = df[df.hours >= 20]

In [16]:
# df = df[~df.model.isin(['16.new2', 'online'])]

In [ ]:
metrics = ['hours','Click/group/auc',  'Click/group/auc2', 'Dur/group/click/time_auc',
           'group/auc', 'group/auc2', 'cold/group/auc', 'auc', 'clickmids/auc', 'cold/auc',
            'hours', 'group/sqrt/auc', 'sqrt/auc', 
           'group/click/time_auc', 'quality/group/click/time_auc', 'rmse', 'click/rmse', 
           'rcr_auc', 'vtime_auc', 'ptime_auc',   
           
          ]
gezi.line(df, x='date', y=metrics, color='model')

In [ ]:
metrics = [
           'group/sqrt/auc', 'sqrt/auc', 
           'group/click/time_auc',
           'quality/group/click/time_auc', 'quality/group/auc',
           'group/top1_click_rate', 'group/top3_click_rate',
           'group/click/top1_dur_rate', 'group/click/top3_dur_rate',
          ]
gezi.line(df, x='date', y=metrics, color='model')

In [ ]:
metrics = [
  'group/first_click_position',
  'group/last_click_position',
  'group/ndcg_click',
  'group/ndcg3_click',
  'group/ndcg3_dur',
  'group/ndcg_dur',
#   'group/ndcg7_click',
#   'group/ndcg14_click',
  'group/click/ndcg_dur',
  'group/click/ndcg3_dur',
#   'group/click/ndcg7_dur',
#   'group/click/ndcg14_dur',
  'quality/group/ndcg_dur',
  'quality/group/click/ndcg_dur',
  'quality/group/click/ndcg3_dur',
]
gezi.line(df, x='date', y=metrics, color='model')